In [ ]:
import os
import sys

sys.path.append(os.path.abspath("src"))

import logging
import sys

import hydra
import mlflow
import numpy as np
import pandas as pd
import torch
from joblib import Memory
from omegaconf import DictConfig, OmegaConf

from evaluators import torchFastTextEvaluator
from framework_classes import (
    DATA_GETTER,
    DATASETS,
    LOSSES,
    MODELS,
    MODULES,
    OPTIMIZERS,
    PREPROCESSORS,
    SCHEDULERS,
    TOKENIZERS,
    TRAINERS,
)
from models import FastTextWrapper
from utils.data import get_df_naf, get_file_system, get_processed_data, get_test_data, get_Y
from utils.mappings import mappings
from utils.mlflow import create_or_restore_experiment
from utils.validation_viz import calibration_curve, confidence_histogram, sort_and_get_pred

%load_ext autoreload
%autoreload 2

In [ ]:
revision = "NAF2008"
model_class = "torchFastText"
start_month = 1
start_year = 2018
text_feature = "libelle"
textual_features_1 = "NAT_LIB"
textual_features_2 = "AGRI"
categorical_features_1 = "TYP"
categorical_features_2 = "NAT"
categorical_features_3 = "SRF"
categorical_features_4 = "CJ"
categorical_features_5 = "CRT"

In [ ]:
cfg_dict = {"data": 
                {"sirene":"sirene_4", 
                "start_month": start_month, 
                "start_year": start_year, 
                "revision": revision,
                "text_feature": text_feature,
                "textual_features" : [textual_features_1, textual_features_2],
                "categorical_features" : [categorical_features_1, categorical_features_2, categorical_features_3, categorical_features_4, categorical_features_5],}, 
                
            "model":{"name": "fastText",
                    "preprocessor": "fastText",
                    "test_params": {"test_batch_size": 256, "run_id":'runs:/45afc22a961a4cdcb282aad93693326d/model'}}
            }
cfg_dict_data = cfg_dict["data"]
df_naf = get_df_naf(revision=cfg_dict_data["revision"])
Y = get_Y(revision=revision)
df_test_ls= get_test_data(**cfg_dict["data"], y=Y)

In [ ]:
Y = get_Y(revision=cfg_dict["data"]["revision"])
df_train, df_val, df_test = get_processed_data()

In [ ]:
import mlflow

logged_model = 'runs:/65bc7a269ea145248476b8c976090784/default'
mlflow.set_tracking_uri("https://projet-ape-mlflow.user.lab.sspcloud.fr/")
# Load model as a PyFuncModel.
fasttext = mlflow.pyfunc.load_model(logged_model)

In [ ]:
fasttext_preds = fasttext.predict(df_test)

In [ ]:
run_id = cfg_dict["model"]["test_params"]["run_id"]
module = mlflow.pytorch.load_model(run_id)

In [ ]:
torch_preds = torchFastTextEvaluator(module).get_preds(
            df=df_test,
            Y=Y,
            **cfg_dict["data"],
            batch_size=cfg_dict["model"]["test_params"]["test_batch_size"],
            num_workers=os.cpu_count() - 1,
        )